In [1]:
%load_ext autoreload
%autoreload 2
import pickle
import numpy as np
import pandas as pd
import numpy as np
import lime
import torch
import torch.nn.functional as F
from lime.lime_text import LimeTextExplainer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
np.random.seed(seed=0)


In [2]:
filename_model = 'roberta-base'
tokenizer = AutoTokenizer.from_pretrained(filename_model)
model = AutoModelForSequenceClassification.from_pretrained(filename_model)

model.load_state_dict(torch.load(
    '/nfs/turbo/coe-vvh/ljr/Censorship/models/20230219-13:48:42_binary_0.1_8_1e-05_adamw.pt'))
model.eval()

class_names = ['uncensored', 'censored']


def predictor(texts):
    outputs = model(**tokenizer(texts, return_tensors="pt", padding=True))
    tensor_logits = outputs[0]
    probas = F.softmax(tensor_logits).detach().numpy()
    return probas

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.

In [ ]:
data = pd.read_csv(
    '/nfs/turbo/coe-vvh/ljr/Censorship/data/dataset_test_p_en.csv', engine='python')


In [ ]:
tmp_df = data.sample(n=1000, replace=False, random_state=1)
# print(tmp_df['censored'].values, tmp_df['full_text_proc'].values)
shap_values = explainer(tmp_df['text_proc'].values.tolist())


In [ ]:
text = ['Building more bypasses will help the environment by reducing pollution and traffic jams in towns and cities.',
        'Poland where political correctness has no place &amp; people speak their minds.\n\nPolish Defense League will NOT allow Invaders to Harm Citizens!'
        ]
print(tokenizer(text, return_tensors='pt', padding=True))

explainer = LimeTextExplainer(class_names=class_names)
exp = explainer.explain_instance(
    text[1], predictor)
exp.show_in_notebook(text=text)


{'input_ids': tensor([[    0, 37500,    55, 15037,   293,    40,   244,     5,  1737,    30,
          4881,  6631,     8,  1703, 28687,    11,  6815,     8,  1947,     4,
             2,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1],
        [    0, 32400,   463,   147,   559, 34726,    34,   117,   317,   359,
          3914,   131,    82,  1994,    49,  7283,     4, 50118, 50118, 32400,
          1173,  4545,   815,    40,  8127,  1157, 47103,     7, 33619, 14047,
           328,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1]])}
